In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [9]:
import logging
logging.basicConfig(level=logging.INFO)
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

### Authenticate

In [12]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]


INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


### Set Pipeline Options and Load External Targets

In [16]:
# # you can use this config and external target list for a minimal test on six targets within one tile...

# cfg_dict = dict(
#     # TODO change this path to wherever you want to save everything
#     base_dir='/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version',
#     name='external_targets_debug',
#     release_name=None, # for no specific release or e.g. 'F-006' for one release 
#     release_priority=['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006'],
#     bands=['VIS', 'NIR_Y'],
#     auxillary_products=['MERPSF','MERRMS','MERBKG'],
    
#     # these will be filled in automatically using on base_dir
#     download_dir=None,
#     tile_dir=None,
#     cutout_dir=None,
#     fits_dir=None,
#     sanity_dir=None
    
# )
# cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
# pipeline.create_folders(cfg)  # fill in those missing paths

# external_targets = pd.DataFrame([
#     {
#       'id_str': 'a',
#       'target_ra': 92.8371844681179,
#       'target_dec': -47.8868920683699,
#       'target_field_of_view': 20},
#      {
#       'id_str': 'b',
#       'target_ra': 92.836691187603,
#       'target_dec': -48.1117628801209,
#       'target_field_of_view': 20},
#      {
#       'id_str': 'd',
#       'target_ra': 92.835560566775,
#       'target_dec': -47.978437467137,
#       'target_field_of_view': 20},
#      {
#       'id_str': 'e',
#       'target_ra': 92.8351734327103,
#       'target_dec': -48.1054532354171,
#       'target_field_of_view': 20},
#      {
#       'id_str': 'f',
#       'target_ra': 92.8343976295701,
#       'target_dec': -47.9993017111107,
#       'target_field_of_view': 20}
    
# ])

INFO:root:Saving to /media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug


In [56]:
release_priority = ['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006']
# release_priority.reverse()

In [73]:
# or you can use this config to run for real

cfg_dict = dict(
    base_dir='/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline',
    name='demo_lrg',
    release_name=None,  # allow tiles from any release
    bands=['VIS', 'NIR_Y', 'NIR_J'],  
    auxillary_products=['MERPSF','MERRMS'],  # ,'MERBKG'
    release_priority=release_priority,
    
    # ignore these
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths


INFO:root:Saving to /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg


{'base_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline', 'name': 'demo_lrg', 'release_name': None, 'bands': ['VIS', 'NIR_Y', 'NIR_J'], 'auxillary_products': ['MERPSF', 'MERRMS'], 'release_priority': ['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612', 'F-006'], 'download_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg', 'tile_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles', 'cutout_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/cutouts', 'fits_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/cutouts/fits', 'sanity_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/sanity'}

In [74]:

# TODO use your own csv, it just needs the columns specified here
# external_targets = pd.read_csv('/media/home/my_workspace/combined_karina_strong_lens_classifications.csv', usecols=['id_str', 'target_ra', 'target_dec'])
# external_targets['target_field_of_view'] = 20  # arcseconds

external_targets = pd.read_csv('/media/home/my_workspace/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/LRGs_4sims_all.csv')
external_targets = external_targets.rename(columns={'ra': 'target_ra', 'dec': 'target_dec', 'ID': 'id_str'})
del external_targets['Unnamed: 0']
external_targets['target_field_of_view'] = 20  # arcseconds

In [75]:
len(external_targets)

15146

In [76]:
external_targets

,id_str,target_ra,target_dec,target_field_of_view
0,EUC_15898_DESI-sel,268.870653,65.253093,20
1,EUC_2767_DESI-sel,150.341459,1.864402,20
2,EUC_2771_DESI-sel,150.364732,1.846961,20
3,EUC_21374_DESI-sel,267.736754,66.934327,20
4,EUC_17153_DESI-sel,272.923035,65.479365,20
...,...,...,...,...
15141,1237671751363723635,180.609610,68.433166,20
15142,1237671751363723635,180.609610,68.433166,20
15143,1237671751363723662,180.794080,68.441164,20
15144,1237671751363723662,180.794080,68.441164,20


### For each target, find a tile that covers it

In [77]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.
INFO:root:Begin target/tile cross-match


INFO: Query finished. [astroquery.utils.tap.core]
Found 2514  query results


INFO:root:Matched 15146 targets to 196 tiles
INFO:root:Targets with tile matches: 14866
/media/user/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/pipeline.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets_with_tiles["tile_index"] = targets_with_tiles["tile_index"].astype(int)


,id_str,target_ra,target_dec,target_field_of_view,tile_index,release_name
0,EUC_15898_DESI-sel,268.870653,65.253093,20,102158890,F-003_240612
1,EUC_2767_DESI-sel,150.341459,1.864402,20,101542818,CALBLOCK_PV-005_R2
2,EUC_2771_DESI-sel,150.364732,1.846961,20,101541378,CALBLOCK_PV-005_R2
3,EUC_21374_DESI-sel,267.736754,66.934327,20,102159774,F-003_240612
4,EUC_17153_DESI-sel,272.923035,65.479365,20,102158894,F-003_240612
...,...,...,...,...,...,...
14861,1237671751363723635,180.609610,68.433166,20,102160541,F-006
14862,1237671751363723635,180.609610,68.433166,20,102160541,F-006
14863,1237671751363723662,180.794080,68.441164,20,102160541,F-006
14864,1237671751363723662,180.794080,68.441164,20,102160541,F-006


In [78]:
targets_with_tiles['release_name'].value_counts()

release_name
F-006                 10884
F-003_240612           2758
CALBLOCK_PV-005_R2     1172
F-003_240321             52
Name: count, dtype: int64

In [79]:
targets_with_tiles = targets_with_tiles.sort_values('tile_index')[:1000]

The speed of the pipeline depends on the number of tiles to download. It takes a few minutes per tile.

In [80]:
targets_with_tiles['tile_index'].nunique()

19

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [ ]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:root:Tile 101541377, 0 of 19
INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 3  query results
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_MOSAIC-NIR-J-RMS_TILE101541377-88F8C6_20240201T185157.303988Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_MOSAIC-NIR-J-RMS_TILE101541377-88F8C6_20240201T185157.303988Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_MOSAIC-NIR-J-RMS_TILE101541377-88F8C6_20240201T185157.303988Z_00.00.fits
INFO:root:Downloaded flux+auxillary tiles: {'VIS': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101541377-2E186_20240201T190014.111492Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_MOSAIC-VIS-RMS_TILE101541377-E3AEDE_20240201T185327.031980Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_CATALOG-PSF-VIS_TILE101541377-B2B5C3_20240201T190536.758003Z_00.00.fits'}, 'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consorti

Done.


INFO:root:Tile 101541378, 1 of 19
INFO:astroquery:Query finished.


...

Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_MOSAIC-NIR-Y-RMS_TILE101544257-D5A34D_20240202T044946.693411Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_MOSAIC-NIR-Y-RMS_TILE101544257-D5A34D_20240202T044946.693411Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE101544257-D5A34D_20240202T044946.693411Z_00.00.fits


Done.
Retrieving data.
Data request: TAPCLIENT=ASTROQUERY&RELEASE=sedm&FILE_NAME=EUC_MER_CATALOG-PSF-NIR-Y_TILE101544257-2C6B30_20240203T043905.910116Z_00.00.fits&RETRIEVAL_TYPE=FILE
------>https
host = easotf.esac.esa.int:443
context = /sas-dd/data
Content-type = application/x-www-form-urlencoded
200 
Reading...


INFO:root:EUC_MER_CATALOG-PSF-NIR-Y_TILE101544257-2C6B30_20240203T043905.910116Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE101544257-2C6B30_20240203T043905.910116Z_00.00.fits
INFO:root:Downloaded flux+auxillary tiles: {'VIS': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101544257-6369BD_20240203T032415.932203Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_MOSAIC-VIS-RMS_TILE101544257-FA3319_20240202T042737.068746Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/demo_lrg/tiles/EUC_MER_CATALOG-PSF-VIS_TILE101544257-F0FB3B_20240203T045009.380663Z_00.00.fits'}, 'NIR_J': {'FLUX': '/media/home/team_workspaces/Euclid-Conso

Done.
